# GENIE Learning Scenarios: Carbon Taxes and R&D

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

from message_ix.tools.add_learning import add_learning


%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
mp = ixmp.Platform("local")

## Making a clone of the existing scenario 'baseline'

Dump scenario to excel nad create a scenario from excel

In [3]:
base = message_ix.Scenario(
    mp, model="GENIE", scenario="learning",
    )

Clone baseline scenario and add learning parameters

## High CO2 Tax with no R&D Target

To add learning parameters to the scenario, one needs to include `add_learning(Scenario)` to the script, with `Scenario` is the name of scenario object that you want to run. In this example, we want to run `scen`, thus:

In [4]:
scen = base.clone(
    "GENIE",
    "learning_CTh-RNDn",
    "high carbon tax with no RND target",
    keep_solution=False,
)
scen.check_out()

In [5]:
all_years = scen.set('year')
all_nodes = scen.set('node')
type_years = list(all_years[all_years>=2020])
nodes = list(all_nodes[all_nodes!="World"])

Base2020 = 10 # $/tCO2
growth = 0.10 # lo: 3%, med: 5%, hi: 10

value_taxes = [round((Base2020*((1+growth)**(x-2020))),2) for x in type_years]

Add CO2 Tax to the scenario

In [6]:
# In case you need to add the new unit to the platform
mp.add_unit("USD/tCO2")
tax_emission = []
for n in nodes:
    data = make_df(
        "tax_emission",
        node=n,
        type_year=type_years,
        type_tec="all",
        unit="USD/tCO2",
        type_emission="TCE",
        value=value_taxes,
        )
    tax_emission.append(data)

tax_emission = pd.concat(tax_emission)

scen.add_par("tax_emission", tax_emission)


In [7]:
scen.commit(comment="GENIE learning with high carbon tax and no RND target")

In [8]:
scen.solve(gams_args =["--learningmode=1"])

In [23]:
#scen.var("OBJ")["lvl"]
#scen.par("emission_factor",filters={'technology':'gas_cc_ccs','emission':'CO2'})
#scen.par("inv_cost",filters={'technology':['gas_cc_ccs','bio_istig_ccs','bio_istig',
#                                           'eth_bio_ccs','h2_bio_ccs','liq_bio_ccs','coal_adv_ccs'],
#                             'year_vtg':2030,
#                             'node_loc':'R11_EEU'})
#scen.var("CAP",filters={'technology':'bio_istig_ccs', 'year_act':2050})

,node_loc,technology,year_vtg,value,unit
0,R11_EEU,bio_istig,2030,3907.69,USD/kWa
1,R11_EEU,bio_istig_ccs,2030,5388.63,USD/kWa
2,R11_EEU,coal_adv_ccs,2030,4129.71,USD/kWa
3,R11_EEU,eth_bio_ccs,2030,3627.11,USD/kWa
4,R11_EEU,gas_cc_ccs,2030,2145.83,USD/kWa
5,R11_EEU,h2_bio_ccs,2030,3445.03,USD/kWa
6,R11_EEU,liq_bio_ccs,2030,3979.37,USD/kWa


## Plotting Results

In [ ]:
from message_ix.reporting import Reporter
from message_ix.util.tutorial import prepare_plots

rep = Reporter.from_scenario(scen)
prepare_plots(rep)

### Activity

How much energy is generated in each time period from the different potential sources?

In [ ]:
rep.set_filters(t=["wind_ppl"])
rep.get("plot activity")

### Capacity

How much capacity of each plant is installed in each period?

In [ ]:
rep.get("plot capacity")

## Close the connection to the database

In [24]:
mp.close_db()